In [ ]:
import os
import gzip
import json
import pandas as pd
pd.options.display.max_columns=1000
import numpy as np
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# 1. merge all data

In [ ]:
matching_dict = {}  #insert matching dict here

In [ ]:
primary_file = 'export_group_assignment_small.csv'
df = pd.read_csv('data/small/export_group_assignment_small.csv')
df = df[matching_dict[primary_file]]

In [ ]:
table_name = '_'.join([x for x in primary_file.split('_') if ('export' not in x) & ('small' not in x)])
df.columns = [table_name + '_' + x if (x != 'group_id') &  (x != 'client_id') else x for x in df.columns]


In [ ]:
#drop duplicated rows

df.drop_duplicates(inplace=True)
df

# 2. feature engineering: stage 1 model

## 2.1 time_diff_group_lead_creation
categorical difference between group_assignment_created_at and client_funnel_lead_created_at
- no diff: 0
- less than 10 min: 1
- more than 10min: 2

In [ ]:
df['group_assignment_created_at'] = df['group_assignment_created_at'].apply(lambda x: pd.to_datetime(x.replace('Z', '').strip()))


In [ ]:
df['client_funnel_lead_created_at'] = df['client_funnel_lead_created_at'].apply(lambda x: pd.to_datetime(str(x).replace('Z', '').strip()))


In [ ]:
df['time_diff_group_lead_creation'] = df['group_assignment_created_at'] - df['client_funnel_lead_created_at']

In [ ]:
#investigate what buckets i should create

timediff_count = df['time_diff_group_lead_creation'].value_counts().reset_index()
timediff_count

In [ ]:
sameday_mask = timediff_count['index'].apply(lambda x: x.days == 0)
sameday_diffhours = timediff_count[sameday_mask]['index'].apply(lambda x: round(x / np.timedelta64(1, 'h')))


In [ ]:
plt.hist(sameday_diffhours, bins=25)

In [ ]:
diffday_mask = timediff_count['index'].apply(lambda x: x.days != 0)
diffday_days = timediff_count[diffday_mask]['index'].apply(lambda x: round(x.days))

In [ ]:
plt.hist(diffday_days, bins=40, range=(0,400))  #zoom the histogram in to investigate further

In [ ]:
def get_timediff_category(x):
    total_seconds = x.total_seconds()
    num_hours = x.total_seconds() / 60 / 60
    num_days = x.days
    
    if np.isnan(total_seconds):
        return 0  #cannot calculate a timediff 
    if total_seconds == 0:  #no timediff at all
        return 1
    else:
        return 2

df['timediff_category_group_lead_creation'] = df['time_diff_group_lead_creation'].apply(lambda x: get_timediff_category(x))

In [ ]:
df['timediff_category_group_lead_creation'].value_counts(normalize=True)

In [ ]:
df.drop('time_diff_group_lead_creation', axis='columns', inplace=True)

## 2.2 client_funnel_first_touch

In [ ]:
import ast

def get_dict(x):
    try:
        return ast.literal_eval(x)
    except:
        return np.nan

df['client_funnel_first_touch_dict'] = df['client_funnel_first_touch'].apply(lambda x: get_dict(x))

In [ ]:
def get_time(x):
    try:
        return pd.to_datetime(x['created_at'].replace('Z', '').strip())
    except:
        return np.nan

df['client_funnel_first_touch_time'] = df['client_funnel_first_touch_dict'].apply(lambda x: get_time(x))


In [ ]:
df['timediff'] = df['client_funnel_lead_created_at'] - df['client_funnel_first_touch_time']  #lead is created after first touch recorded!


In [ ]:
def get_days(x):
    try:
        return round(x.days)
    except:
        return np.nan

days = df['timediff'].apply(lambda x: get_days(x))

In [ ]:
days.value_counts(normalize=True)

In [ ]:
plt.hist(days, range=(-100,100), bins=70)

## 2.3 timediff_firsttouch_leadcreation
time difference between first touch timestamp and lead creation timestamp
- no timediff data avai: 0
- more than 1 day before: 1
- 1 day before: 2
- same day: 3
- after same day: 4

In [ ]:
def get_category(x):
    try:
        day = round(x.days)
    except:
        return 0
    if day < -1:
        return 1
    if day == -1:
        return 2
    if day == 0:
        return 3
    else:
        return 4

df['timediff_firsttouch_leadcreation'] = df['timediff'].apply(lambda x: get_category(x))

In [ ]:
df['timediff_firsttouch_leadcreation'].value_counts(normalize=True)

## 2.4. client_funnel_first_touch_medium
medium on which the first touch took place

In [ ]:
def get_medium(x):
    try:
        return x['category']['description']
    except:
        return np.nan

df['client_funnel_first_touch_medium'] = df['client_funnel_first_touch_dict'].apply(lambda x: get_medium(x))

In [ ]:
df['client_funnel_first_touch_medium'].value_counts(normalize=True)

In [ ]:
#drop all intermediate cols created

In [ ]:
df.drop(['client_funnel_first_touch_dict', 'client_funnel_first_touch_time', 'timediff'], axis='columns', inplace=True)


## 2.5. client_funnel_layout_type

In [ ]:
df['client_funnel_layout_type'].unique()

In [ ]:
def clean_layout(x):
    if type(x) != str:
        return 'Unknown'
    if x == '(Unknown)':
        return 'Unknown'
    else:
        return x

df['client_funnel_layout_type'] = df['client_funnel_layout_type'].apply(lambda x: clean_layout(x))

In [ ]:
df['client_funnel_layout_type'].value_counts(normalize=True)

## 2.6. client_funnel_client_lead_source

In [ ]:
import ast

def get_dict(x):
    try:
        return ast.literal_eval(x)
    except:
        return np.nan

df['client_funnel_client_lead_source_dict'] = df['client_funnel_client_lead_source'].apply(lambda x: get_dict(x))
df['client_funnel_client_discovery_source_dict'] = df['client_funnel_client_discovery_source'].apply(lambda x: get_dict(x))
df['client_funnel_client_origin_source_dict'] = df['client_funnel_client_origin_source'].apply(lambda x: get_dict(x))


In [ ]:
def get_top_lead(cell):
    try:
        if cell['description'] == 'Apartments.com':
            return 'Apartments.com'
        elif cell['description'] == 'Google':
            return 'Google'
        elif cell['description'] == 'Manual':
            return 'Manual'
        elif cell['description'] == 'Website':
            return 'Website'
        elif cell['description'] == 'Apartmentlist':
            return 'Apartmentlist'
        elif cell['description'] == 'Google My Business':
            return 'Google My Business'
        else:
            return 'Other'
    except:
        return 'Other'

def get_description(cell):
    try:
        return cell['description']
    except:
        return 'Unknown'
    
df['client_top_lead_source'] = df['client_funnel_client_lead_source_dict'].apply(lambda x: get_top_lead(x))
df['client_all_lead_source'] = df['client_funnel_client_lead_source_dict'].apply(lambda x: get_description(x))

In [ ]:
def get_top_discovery(cell):
    try:
        if cell['description'] == 'Apartments.com':
            return 'Apartments.com'
        elif cell['description'] == 'Google':
            return 'Google'
        elif cell['description'] == 'Website':
            return 'Website'
        elif cell['description'] == 'Resident Referral':
            return 'Resident Referral'
        elif cell['description'] == 'Zumper/Padmapper':
            return 'Zumper/Padmapper'
        elif cell['description'] == 'Craigslist':
            return 'Craigslist'
        else:
            return 'Other'
    except:
        return 'Other'

def get_description(cell):
    try:
        return cell['description']
    except:
        return 'Unknown'
    
df['client_top_discovery_source'] = df['client_funnel_client_discovery_source_dict'].apply(lambda x: get_top_discovery(x))
df['client_all_discovery_source'] = df['client_funnel_client_discovery_source_dict'].apply(lambda x: get_description(x))

In [ ]:
def get_top_origin(cell):
    try:
        if cell['description'] == 'Syndication Phone':
            return 'Syndication Phone'
        elif cell['description'] == 'Website - Appt Schedule':
            return 'Website - Appt Schedule'
        elif cell['description'] == 'Manual':
            return 'Manual'
        elif cell['description'] == 'Apartments.com':
            return 'Apartments.com'
        elif cell['description'] == 'Call Center':
            return 'Call Center'
        elif cell['description'] == 'Website':
            return 'Website'
        else:
            return 'Other'
    except:
        return 'Other'

def get_description(cell):
    try:
        return cell['description']
    except:
        return 'Unknown'
    
df['client_top_origin_source'] = df['client_funnel_client_origin_source_dict'].apply(lambda x: get_top_origin(x))
df['client_all_origin_source'] = df['client_funnel_client_origin_source_dict'].apply(lambda x: get_description(x))

In [ ]:
df.drop(['client_funnel_client_lead_source_dict', 'client_funnel_client_discovery_source_dict', 
         'client_funnel_client_origin_source_dict'], axis=1, inplace=True)

# 3. clean target

In [ ]:
df['client_funnel_is_rented'].value_counts(normalize=True)

#IMPT: target is skewed!

In [ ]:
#drop nulls in target

df['client_funnel_is_rented'].isna().sum() / len(df)

In [ ]:
df = df[~df['client_funnel_is_rented'].isna()]

# 3. feature engineering: stage 2 model

## 3.1. avg_convo_text_length
= average text length across all recorded conversations

In [ ]:
def get_length(x):
    try: 
        if np.isnan(x):
            return np.nan
    except:
        return len(x)

df2['convo_text_length'] = df2['conversations_message_text'].apply(lambda x: get_length(x))

In [ ]:
df2['avg_convo_text_length'] = df2.groupby(['client_id', 'group_id'])['convo_text_length'].transform(lambda x: x.sum() / len(x))


In [ ]:
df2['avg_convo_text_length'].value_counts(normalize=True)

In [ ]:
df2.drop(['conversations_message_text', 'convo_text_length'], axis='columns', inplace=True)

## 3.2. num_conversations

In [ ]:
num = df2.groupby(['client_id', 'group_id']).transform(lambda d: len(d))

In [ ]:
df2['num_conversations'] = num.iloc[:, 0]

In [ ]:
#account for rows that have nulls for conversation variables: num_conversations should be 0 but they are non-zero from the above len() method

no_conversation_indexes = df2[df2['conversations_medium'].isnull()].index


In [ ]:
df2.loc[no_conversation_indexes, 'num_conversations'] = 0

In [ ]:
df2['num_conversations'].value_counts(normalize=True)

## create vars: count of each medium used for convo, count of all mediums used, count of each direction

In [ ]:
import collections

def get_counts(row):
    medium_counts = collections.Counter(row['conversations_medium'])
    direction_counts = collections.Counter(row['conversations_direction'])
    return (medium_counts, direction_counts)
    
counts = df2.groupby(['client_id', 'group_id']).apply(lambda d: get_counts(d))

In [ ]:
counts = counts.reset_index()
counts = counts.rename(columns = {0: 'counts'})

In [ ]:
counts

In [ ]:
df2.drop(['conversations_medium', 'conversations_direction'], axis = 'columns', inplace=True)
df2.drop_duplicates(inplace=True)

In [ ]:
df2

In [ ]:
df2 = df2.merge(counts, how='left', on=['client_id', 'group_id'])

In [ ]:
#clean 'counts' col

def clean_counts(x):
    medium_counts = {'phone': 0, 'email': 0, 'sms': 0, 'other': 0}
    direction_counts = {'incoming': 0, 'outgoing': 0}
    num_total_mediums = 0
    
    medium_dict = x[0]
    direction_dict = x[1]
    
    for key, val in medium_dict.items():
        if key == 'Phone':
            medium_counts['phone'] = val
            num_total_mediums += 1
        elif key == 'Email Message':
            medium_counts['email'] = val
            num_total_mediums += 1
        elif key == 'SMS Message':
            medium_counts['sms'] = val
            num_total_mediums += 1
        elif key == 'Other Message Type':
            medium_counts['other'] = val
            num_total_mediums += 1
    
    for key, val in direction_dict.items():
        if key == 'Incoming':
            direction_counts['incoming'] = val
        elif key == 'Outgoing':
            direction_counts['outgoing'] = val
        
    return num_total_mediums, medium_counts['phone'], medium_counts['email'], medium_counts['sms'], medium_counts['other'], direction_counts['incoming'], direction_counts['outgoing']

df2['num_total_mediums'], df2['medium_phone_count'], df2['medium_email_count'], df2['medium_sms_count'], df2['medium_other_count'], df2['direction_incoming_count'], df2['direction_outgoing_count'] = zip(*df2['counts'].map(clean_counts))
        

In [ ]:
df2.drop('counts', axis='columns', inplace=True)

# 4. build model

## 4.1. CatBoost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  auc, roc_curve, precision_recall_curve, accuracy_score

In [ ]:
df_stage1 = df[['group_assignment_company_id', 'client_funnel_layout_type', 'client_funnel_community_name', 
                'client_top_lead_source', 'client_top_discovery_source', 'client_top_origin_source', 
                'client_all_lead_source', 'client_all_discovery_source', 'client_all_origin_source',
                'client_funnel_new_lead_group', 'client_funnel_is_walk_in', 
                'timediff_firsttouch_leadcreation', 'client_funnel_first_touch_medium', 
                'timediff_category_group_lead_creation', 'client_funnel_is_rented']]


In [ ]:
df_stage1.dropna(inplace=True)

In [ ]:
df_stage1['client_funnel_is_walk_in'] = df_stage1['client_funnel_is_walk_in'].apply(lambda x: int(x))
df_stage1['client_funnel_new_lead_group'] = df_stage1['client_funnel_new_lead_group'].apply(lambda x: int(x))


In [ ]:
categorical_f = []  #insert categorical feature names here

In [ ]:
X = df_stage1.drop('client_funnel_is_rented', axis=1)
y = df_stage1["client_funnel_is_rented"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### 4.1.1 tune CatBoost using Bayesian Optimization

In [ ]:
catboost = CatBoostClassifier(loss_function='Logloss', cat_features=categorical_f, one_hot_max_size=7, verbose=10)
# one_hot_max_size = 7 so that layout_type, new_lead_group, and other columns with many levels will be processed using CatBoost's integrated target statistics

param = {
    'iterations':Integer(100, 250), # on the low side to speed up computation (learning rate will adjust accordingly)
    'depth':Integer(1, 10),
    'random_strength':Real(1e-9, 10), # amount of randomness to use for scoring splits (used to prevent overfitting)
    #'bagging_temperature':Real(0.0, 1.0),
    'l2_leaf_reg':Real(0.001, 10000), # coefficient at the L2 regularization term (lambda)
    'scale_pos_weight':Real(1, 50), # weight for class 1 in binary classification
    'subsample':Real(0.5, 1),
    'colsample_bylevel':Real(0.5,1),
    'model_size_reg':Real(0.01, 1000), # model size regularization coefficient
    'leaf_estimation_iterations':[1,5] # how many steps are done in every tree when calculating leaf values (values recommendated in documentation)
}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True) 
    
opt = BayesSearchCV(catboost, param, scoring = LogLoss, n_iter=20, cv=3, random_state=101, verbose=1)

# executes bayesian optimization
opt.fit(X_train, y_train)


In [ ]:
opt.best_params_

In [ ]:
# replace the best parameters in this dictionary
best_para = {
    'iterations': 250,
    'depth': 9,
    'random_strength': 1e-9,
    'l2_leaf_reg': 0.001,
    'scale_pos_weight':1.0,
    'subsample':1.0,
    'colsample_bylevel':0.5,
    'model_size_reg': 0.01,
    'leaf_estimation_iterations': 5
}

In [ ]:
catboost = CatBoostClassifier(**best_para, cat_features=categorical_f, loss_function='Logloss', one_hot_max_size=7)
catboost.fit(X_train, y_train, verbose=30)
proba_cat = catboost.predict_proba(X_test)[:, 1]
y_pred=catboost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test, proba_cat)
auc_ = auc(fpr, tpr)
precision, recall, thresholds_2 = precision_recall_curve(y_test, proba_cat)
auc_2 = auc(recall, precision)

In [ ]:
plt.title('Preliminary', fontsize=20)
plt.plot(recall, precision, 'b', label = 'AUC = %0.2f' % auc_2, color='#ff00ff', linewidth=3)
plt.legend(loc = 'lower right')
plt.plot([0, 1],[0.158,0.158], color='#ffffff', linestyle='--', linewidth=3)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Precision', fontsize=14)
plt.xlabel('Recall', fontsize=14)
plt.yticks(size = 14)
plt.xticks(size = 14)
plt.style.use('seaborn')
plt.show()
plt.tight_layout()

### 4.1.2 get permutation feature importances

In [ ]:
importance = catboost.get_feature_importance(prettified=True, thread_count=-1, verbose=False).set_index('Feature Id')
plt.rcParams["figure.figsize"] = (12,7)
ax = importance.plot(kind='barh', color='#ff00ff')
plt.style.use('seaborn-bright')


In [ ]:
def permutation_importances(model, X, y, metric):
    baseline = metric(model, X, y)
    imp = []
    for col in X.columns:
        save = X[col].copy()
        X[col] = np.random.permutation(X[col])
        m = metric(model, X, y)
        X[col] = save
        imp.append(m-baseline)
    return np.array(imp)

def get_feature_imp_plot(model, method):
    
    fi =  permutation_importances(model, X_test, y_test, LogLoss)
    feature_score = pd.DataFrame(list(zip(X_test.dtypes.index, fi )),
                                    columns=['Feature','Score'])

    feature_score = feature_score.sort_values(by='Score', ascending=False, inplace=False, kind='quicksort', na_position='last')

    plt.rcParams["figure.figsize"] = (12,7)
    ax = feature_score.plot('Feature', 'Score', kind='barh', color='c')
    ax.set_title("Feature Importance using {}".format(method), fontsize = 14)
    ax.set_xlabel("features")
    plt.show()

In [ ]:
%time get_feature_imp_plot(catboost, method="Permutation")

### 4.1.3 get SHAP feature importances

In [ ]:
import shap

In [ ]:
explainer = shap.Explainer(catboost)
shap_values = explainer(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, use_log_scale=True)

#X-axis: log odds of renting

In [ ]:
shap.plots.bar(shap_values, max_display=100)

## 4.2. logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logr = LogisticRegression()
logr.fit(x_train, y_train)
proba_li = logr.predict_proba(x_test)[:, 1]
y_pred = logr.predict(x_test)

## 4.3. XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(max_depth=3,eta=0.1, colsample_bytree = 0.8,subsample=0.8)
xgb.fit(x_train, y_train)
proba_xgb = xgb.predict_proba(x_test)[:, 1]
y_pred = xgb.predict(x_test)

### 4.3.1. get permutation feature importances

In [ ]:
feature_important = xgb.get_booster().get_score(importance_type='permutation')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.plot(kind='barh')
